# Bethe roots VQE for the closed XXZ chain: arXiv:2404.18244
## David Raveh* and Rafael I. Nepomechie**
## * david.raveh@rutgers.edu  $\quad$ ** nepomechie@miami.edu

It is assumed here that Qiskit has already been installed (pip install qiskit), as well as qiskit-aer, qiskit-algorithms, and qiskit-nature. See 

https://docs.quantum.ibm.com/guides/install-qiskit

In [ ]:
import qiskit

qiskit.version.get_version_info()

## Function $f(w,\vec k)$ for the *closed* XXZ chain

Make the closed-chain function $f(w,\vec k)$ for a given set of bethe roots $\vec k=[k_1,\dots,k_M]$

$$f(w, \vec{k}) = \sum_{\sigma\in S_M} \varepsilon(\sigma)\, A(k_{\sigma(1)}, \cdots, k_{\sigma(M)})\, 
    e^{i \sum_{j=1}^M k_{\sigma(j)} x_j} $$

Import the necessary libraries

In [1]:
import numpy as np
import math
import itertools 
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)

In [2]:
#mylist is a collection of permutations t of the numbers 0 to M-1
def mylist(M): 
    return list(itertools.permutations(range(M)))

# inversion number of t=(0,1,2,...,M-1)
def inv(t):
    x=0
    for j in range(M):
        for i in range(j):
            if t[i]>t[j]:
                x=x+1
    return x

# sign of permutation t
def e(t):
    return (-1)**inv(t)
    
def s(k,kp):
    return 1-2*Delta*np.exp(1.j*kp)+np.exp(1.j*(k+kp))

# k=[k0,...,kM-1]
def A(k):
    x=1
    for l in range(len(k)):
        for j in range(l):
            x=x*s(k[l],k[j])
    return x

# permutes the bethe roots k into an order determined by t: [k0,k1,k2] -> [k1,k2,k0]
def kperm(k,t):
    y=[]
    for i in t:
        y.append(k[i])
    return y

# positions of the ones in w
def positions(w):
    x=[]
    for i in range(len(w)):
        if w[i]==1:
            x.append(i+1) # positions run from 1 to L, not 0 to L-1
    return x

def f(w,k):
    x=positions(w)
    z=0
    for t in mylist(len(k)):
        y=kperm(k,t)
        v=0
        for i in range(len(k)):
            v=v+y[i]*x[i]
        v=np.exp(1.j*v)
        z=z+e(t)*A(y)*v
    return z

## Hamiltonian

We now create the operators $\mathcal H$ (ham) and $\mathcal H ^2$ (hamsquared), a sum of Pauli strings, that will be used in our VQE, with

$$ \mathcal{H} = \tfrac{1}{4}\sum_{n=1}^L 
    \left(\sigma^x_n \sigma^x_{n+1} + \sigma^y_n \sigma^y_{n+1} + \Delta\, \sigma^z_n \sigma^z_{n+1} \right) \,, \qquad \vec{\sigma}_{L+1} = \vec{\sigma}_1 $$

We set $L=4, M=2, \Delta=2$ as an example of how to use this code, and will demonstrate how to generate the ground state and excited states with $M=2$ Bethe roots.

In [3]:
L=4
M=2
Delta=2

In [4]:
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
)
from qiskit_nature.second_q.hamiltonians import HeisenbergModel

from qiskit_nature.second_q.mappers import LogarithmicMapper

In [5]:
num_nodes=L
hamop = HeisenbergModel(LineLattice(num_nodes, boundary_condition=BoundaryCondition.PERIODIC), (1.0, 1.0, Delta), (0.0, 0.0, 0.0))
hamop.second_q_op()
logmapper = LogarithmicMapper()
ham=logmapper.map(hamop.second_q_op())
hamsquared =  ham @ ham

In [50]:
#ham.to_matrix()

Prints the minimum eigenvalue of our Hamiltonian

In [10]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=ham)
ref_value = result.eigenvalue.real
print(ref_value)

-2.7320508075688803


## Variational ansatz

In [6]:
from qiskit.circuit import ParameterVector

kparts represents the set of parameters. Of the "kparts" cells, only one should be run; if you are searching for all real roots, the first cell is run, etc.

For the ground state of $L=4$, we search for two real roots, i.e. we run the first cell. For excited states, we search for roots that are complex conjugate pairs, i.e. we run the second cell.

k represents the set of Bethe roots, $\vec k=[k_1,\dots,k_M]$, allowing $k_i$ to be complex numbers

In [40]:
# M real roots
kparts = ParameterVector("kparts", M)
k=kparts

In [7]:
# one complex conjugate pair, M=2
kparts = ParameterVector("kparts", 2)
k=[kparts[0]+1j*kparts[1],kparts[0]-1j*kparts[1]]

In [12]:
# one complex pair, one real root, M=3
kparts = ParameterVector("kparts", 3)
k=[kparts[0]+1j*kparts[1],kparts[0]-1j*kparts[1],kparts[2]]

## Bethe Circuit

Load the necessary qiskit libraries

In [8]:
from qiskit import *
from qiskit import QuantumRegister
from qiskit import QuantumCircuit
from qiskit.circuit.library import RYGate
from qiskit.circuit.library import UGate
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)

bethe denotes the quantum circuit, or ansatz, that will be used in our VQE. 

$$ |B^L_M (\vec{k})\rangle= \sum_{w\in P(L,M)}f(w, \vec{k})\,|w\rangle $$

The angles in the unitaries in the circuit are functions of $\vec k$, the set of parameterized Bethe roots. Note that currently there is a bug in qiskit that doesn't allow creating a parameterized unitary with 4 or more controls ( https://github.com/Qiskit/qiskit/issues/12135 ), which prevents going beyond $M=5$.

In [9]:
q=QuantumRegister(L)
bethe = QuantumCircuit(q)

# Create set of permutations
def P(L,M):
    if M==0:
        return [[0] * L]
    elif L-M==0:
        return [[1] * L]
    else:
        x=[]
        for p0 in P(L-1,M):
            x.append(p0+[0])
        for p1 in P(L-1,M-1):
            x.append(p1+[1])    
        return x
    
# store the coefficients f(w,k)
farray=[]
for w in P(L,M):
    farray=farray+[[w,f(w,k)]]
    
def F(b):
    sol=[] # solutions of w=ab
    for w in P(L,M):
        if w[L-len(b):L]==b:
            sol.append(w)
    if len(sol)==1:
        position=P(L,M).index(sol[0])
        return farray[position][1]
    else:  
        x=0
        for p in sol:
            position=P(L,M).index(p)
            x=x+abs(farray[position][1])**2
        return x**0.5

def G(i,b):
    x=[i] + b
    return F(x)/F(b)

# a way of obtaining the angle arg(x) when x is a ParameterExpression
def myangle(x):
    eps=1e-16j
    y = -1j*np.log(x/np.abs(x)+eps)
    return (y+np.conjugate(y))/2

def theta(b):
    return 2*np.arccos(abs(G(1,b)))
def lam(b):
    return myangle(G(0,b)) -np.pi
def phi(b):
    return myangle(G(1,b))-lam(b)

def u(b):
    t=theta(b)
    la=lam(b)
    p=phi(b)
    return UGate(t,p,la)

def applyX(m,l):
    bethe.cx(L-m,L-m+l)

def applyI(m,l):
    sol=[] # these will be the strings b in P(L-m,M-l)
    for b in P(L-m,M-l):
        if F(b)!=0: # Check that F(b)!=0
            sol.append(b)
    if len(sol)==1:
        applyX(m,l)
        b=sol[0] # when b is unique, no extra controls are needed
        if l==1:
            cU = u(b).control(1)
            bethe.append(cU,[L-m+l,L-m])
        else:
            ccU = u(b).control(2)
            bethe.append(ccU,[L-m+l-1,L-m+l,L-m])
        applyX(m,l)
    elif len(sol)>1:
        applyX(m,l)
        for b in sol:
            extra=[] # these are going to be the locations of the extra controls
            for i in range(len(b)):
                if b[i]==1:
                    # if b=[0,1] then extra=[0], location of control
                    extra.append(len(b)-i-1)
            if l==1:
                cU = u(b).control(1+len(extra))
                bethe.append(cU,extra + [L-m+l,L-m])
            else:
                ccU = u(b).control(2+len(extra))
                bethe.append(ccU,extra + [L-m+l-1,L-m+l,L-m])
        applyX(m,l)   

# Create initial state        
for i in range(M):
    bethe.x(i) 
bethe.barrier()
    
# Apply I-gates
for m in reversed(range(2,L+1)):
    for l in range(max(M+m-L,1),min(M,m-1)+1):
        print(m,l)
        applyI(m,l)       
    bethe.barrier()
        
#bethe.draw(output='mpl', initial_state=True)

4 2
3 1
3 2
2 1


## Statevector VQE: Ground state

We now run the ground-state VQE exactly, i.e., the expectation $\langle B^L_M (\vec{k}) | \mathcal{H} | B^L_M (\vec{k})\rangle$ is computed via the Statevector simulator. The VQE is optimized with Cobyla with a maximum of 1000 iterations. Note that the output is kparts, not k (!).

For the example of $L=4$, the output below gives the energy ('eigenvalue') $E=-2.73205$, and the Bethe roots  ('optimal_parameters') $\vec k=[1.94561498, -1.94556757]$.

In [10]:
from qiskit.primitives import Estimator 
from qiskit_algorithms.utils import algorithm_globals
#from qiskit_aer.primitives import Estimator as AerEstimator

estimator = Estimator()

# define optimizer
from qiskit_algorithms.optimizers import COBYLA
iterations = 1000
cobyla= COBYLA(maxiter=1000)

from qiskit_algorithms import VQE
# define callback
# note: Re-run this cell to restart lists before training
counts = []
values = []

def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

In [44]:
x0=np.random.uniform(-np.pi,np.pi,bethe.num_parameters)

print(x0)

vqe = VQE(estimator, bethe, cobyla, callback=store_intermediate_result, initial_point=x0)
result = vqe.compute_minimum_eigenvalue(operator=ham)

print(result)

[ 0.35033641 -0.790704  ]
{   'aux_operators_evaluated': None,
    'cost_function_evals': 43,
    'eigenvalue': -2.7320507882871965,
    'optimal_circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x00000260D3F44190>,
    'optimal_parameters': {   ParameterVectorElement(kparts[0]): 1.9456149774116025,
                              ParameterVectorElement(kparts[1]): -1.9455675690983072},
    'optimal_point': array([ 1.94561498, -1.94556757]),
    'optimal_value': -2.7320507882871965,
    'optimizer_evals': None,
    'optimizer_result': <qiskit_algorithms.optimizers.optimizer.OptimizerResult object at 0x00000260D3C11C40>,
    'optimizer_time': 1.63838529586792}


## Shots VQE: Ground state

We now run the ground-state VQE using shots, i.e., the expectation $\langle B^L_M (\vec{k}) | \mathcal{H} | B^L_M (\vec{k})\rangle$ is approximated via the Aer simulator with 10000 shots. The VQE is optimized with Cobyla with a maximum of 1000 iterations. Note that the output is kparts, not k (!)

For the example of $L=4$, the output gives the energy ('fun') $E=-2.7029$, and the Bethe roots ( 'x') $\vec k=[2.01950078, -2.00694743]$ 

In [54]:
from qiskit_aer.primitives import Estimator as AerEstimator

aer_estimator = AerEstimator(run_options={"shots": 10000})
    
def evaluate_energy(x0):
    estimator_job1 = aer_estimator.run(bethe.assign_parameters({k: x0}), ham)
    estimator_result1  = estimator_job1.result()
    return estimator_result1.values

In [55]:
from qiskit_algorithms.optimizers import COBYLA

cobyla = COBYLA()

x0=np.random.uniform(-np.pi,np.pi,bethe.num_parameters)
print(x0)
opt_result = cobyla.minimize(evaluate_energy, x0)
print(opt_result)

[ 3.10871429 -2.5422158 ]
{   'fun': -2.7029,
    'jac': None,
    'nfev': 31,
    'nit': None,
    'njev': None,
    'x': array([ 2.01950078, -2.00694743])}


## Statevector VQE: Variance

We now run excited-state VQE exactly, i.e., the expectation $\langle B^L_M (\vec{k}) | \mathcal{H}^2 | B^L_M (\vec{k})\rangle-\langle B^L_M (\vec{k}) | \mathcal{H} | B^L_M (\vec{k})\rangle^2$ is computed via the Statevector simulator. The VQE is optimized with Cobyla with a maximum of 1000 iterations. Note that the output is kparts, not k (!)

For the example $L=4, M=2$, with a complex conjugate pair of Bethe roots, we obtain the variance ('fun') 1.9361e-06 $\approx 0$, i.e. an eigenstate. The output parameters ('x') are $[0.78442134, 0.88006142]$, and so the Bethe roots are $[0.78442134+0.88006142i,0.78442134-0.88006142i]$. Running it again may give a different set of Bethe roots that come in a complex conjugate pair.

In [35]:
# define Estimator for exact statevector simulation
from qiskit.primitives import StatevectorEstimator as Estimator

estimator = Estimator()

# define optimizer
from qiskit_algorithms.optimizers import COBYLA
iterations = 1000
cobyla= COBYLA(maxiter=1000)

from qiskit_algorithms import VQE

In [38]:
def evaluate_var(params):
    estimator_job1 = estimator.run([(bethe, hamsquared, params)])
    estimator_job2 = estimator.run([(bethe, ham, params)])
    estimator_result1  = estimator_job1.result()
    estimator_result2  = estimator_job2.result()
    er1=estimator_result1[0].data.evs
    er2=estimator_result2[0].data.evs
    var =  er1  -  er2**2
    return var

x0=np.random.uniform(-np.pi,np.pi,bethe.num_parameters)
print(x0)
opt_result = cobyla.minimize(evaluate_var, x0)
print(opt_result)

[-0.34642509 -0.05449785]
{   'fun': 1.9361153528700894e-06,
    'jac': None,
    'nfev': 122,
    'nit': None,
    'njev': None,
    'x': array([0.78442134, 0.88006142])}


## Shots VQE: Variance

We now run excited-state VQE using shots, i.e., the expectation $\langle B^L_M (\vec{k}) | \mathcal{H}^2 | B^L_M (\vec{k})\rangle-\langle B^L_M (\vec{k}) | \mathcal{H} | B^L_M (\vec{k})\rangle^2$ is approximated via the Aer simulator with 10000 shots. The VQE is optimized with Cobyla with a maximum of 1000 iterations. Note that the output is kparts, not k (!)

For the example $L=4, M=2$, with a complex conjugate pair of Bethe roots, we obtain the variance ('fun') 0.06568 $\approx 0$, i.e. an eigenstate. The output parameters ('x') are $[-0.0061239 ,  0.78056987]$, and so the Bethe roots are $\vec k=[ \pm 0.78056987i]$ (the true roots are $\vec k=[ \pm 0.831443i]$). Running it again may give a different set of Bethe roots that come in a complex conjugate pair.

In [28]:
from qiskit_aer.primitives import Estimator as AerEstimator

aer_estimator = AerEstimator(run_options={"shots": 10000})
    
def evaluate_var(x0):
    estimator_job1 = aer_estimator.run(bethe.assign_parameters({kparts: x0}), hamsquared)
    estimator_result1  = estimator_job1.result()
    estimator_job2 = aer_estimator.run(bethe.assign_parameters({kparts: x0}), ham)
    estimator_result2  = estimator_job2.result()
    er1=estimator_result1.values[0]
    er2=estimator_result2.values[0]
    var =  er1  -  er2**2
    return var

In [32]:
from qiskit_algorithms.optimizers import COBYLA

cobyla = COBYLA()

x0=np.random.uniform(-np.pi,np.pi,bethe.num_parameters)
print(x0)
opt_result = cobyla.minimize(evaluate_var, x0)
print(opt_result)

[-0.87596415  1.18880828]
{   'fun': 0.06568379000000024,
    'jac': None,
    'nfev': 37,
    'nit': None,
    'njev': None,
    'x': array([-0.0061239 ,  0.78056987])}
